In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
%matplotlib inline
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Système de Recommandation de Musique Spotify

## Importation de données

Les données sont d'abord importées et enregistrées dans 3 "dataframes" différents : df_S, df_A et df_G. Dans chacun d'eux, on dispose d'informations techniques en fonction des chansons, des artistes et des genres respectivement. Ensuite, le genre "[]" (qui correspond au vide) est supprimé de df_A et df_G.

In [ ]:
df_S = pd.read_csv("../input/spotify-dataset-19212020-160k-tracks/data.csv")
df_G = pd.read_csv("../input/spotify-dataset-19212020-160k-tracks/data_by_genres.csv")
df_A = pd.read_csv("../input/spotify-dataset-19212020-160k-tracks/data_w_genres.csv")
df_G = df_G.drop(index=1)
df_A = df_A[df_A['genres']!='[]']
df_A = df_A.reset_index().drop(['index'], axis = 1)

Une colonne "genres" est ensuite ajoutée à df_S, assignant à chaque chanson une liste vide.

In [ ]:
genres1 = []
for i in range(len(df_S.index)):
    genres1.append([])

df_S['genres'] = genres1

In [ ]:
df_S.head()

In [ ]:
df_A.head()

In [ ]:
df_G.head()

## Traitement des données

Ensuite, on travaille sur les données des artistes dans df_S et df_A : leurs noms sont standardisés. Dans df_A, les espaces sont supprimés et les guillemets dans les noms des artistes sont fixes, tandis que dans df_S, la même chose est faite, mais en plus, les noms des artistes participant à une chanson sont également entrés dans une liste (pour chaque chanson, les données sont transformées comme : "[x,y,z]" -> ["x","y","z"]).

Ces données sont stockées dans des listes : artistList pour les artistes df_A et songArtist pour les artistes de chaque chanson df_S (tout cela en gardant le même ordre que leurs "dataframes").

En outre, il arrive aussi qu'il y ait plus d'artistes dans df_S (qui participent aux chansons) que dans df_A (naturellement, puisqu'il y a beaucoup d'artistes qui participent seulement aux chansons mais qui n'ont pas leurs propres chansons ou ne sont simplement pas enregistrés). Pour remédier à cela, à partir des listes ci-dessus, une liste appelée AbsentArtistes est créée qui correspond aux artistes de df_S qui ne sont pas enregistrés dans df_A. 

In [ ]:
artistList = df_A['artists'].tolist()
for i in range(len(artistList)):
    artistList[i] = artistList[i].replace(" ","")
    
songArtists0 = df_S['artists'].tolist()
for i in range(len(songArtists0)):
    songArtists0[i] = songArtists0[i].replace(" ","")
    
songArtists = []
for i in range(len(songArtists0)):
    artist = songArtists0[i].replace('[','').replace(']','').split(',')
    for j in range(len(artist)):
        artist[j] = artist[j].replace("'","")
    songArtists.append(artist)
    
absentArtists = []    
for i in range(len(songArtists)):
    for j in range(len(songArtists[i])):
        if not(songArtists[i][j] in artistList):
            absentArtists.append(songArtists[i][j])


Ensuite, à partir de ce qui précède, onpeut créer la liste presentArtist qui correspond à la liste où chaque élément correspond à la liste des artistes participant à une chanson (pour toutes les chansons en df_S), sauf que on ne garde que les artistes qui sont en df_A.

Cet algorithme est très coûteux car il a un ordre de : (longueur des chansons (160k+) * longueur de la liste des artistes de chaque chanson (~[1;4]) * longueur des absentArtist), donc pour voir sa progression est ajouté une impression de l'index qui exécute l'algorithme.

In [ ]:
df_S['artists'] = songArtists

presentArtist = []
for i in range(len(df_S)):
    print(i)
    artsong = df_S.iloc[i,1]
    for j in range(len(artsong)):
        realArtists = []
        if not(artsong[j] in absentArtists):
            realArtists.append(artsong[j])
    presentArtist.append(realArtists)
A = presentArtist    
        
            

Ensuite, la colonne "artistes" de df_S est mise à jour avec la liste presentArtist nouvellement créée et toutes les chansons qui n'ont plus d'artistes sont supprimées.

In [ ]:
df_S['artists'] = A
new_df_S = df_S
L = df_S['artists'].tolist()
for i in range(len(df_S)):
    print(i)
    if L[i] == []:
        new_df_S = new_df_S.drop(index=i)

Les citations sont supprimées des noms dans la liste des artistes et la colonne des noms d'artistes dans df_A est mise à jour.

In [ ]:
artistList2 = []
for i in range(len(artistList)):
    artistList2.append(artistList[i].replace("'",""))
df_A['artists'] = artistList2

Une fonction auxilaire est créée pour joindre les listes sans répéter les éléments.

In [ ]:
# union(A,B): list(any) , list(any) -> list(any)
# elle reçoit deux listes et les réunit sans dupliquer les éléments qui se trouvent sur les deux listes.
# ex: L1 = [1,2,3], L2 = [2,3,4] -> union(L1,L2) = [1,2,3,4]
def union(A,B):
    C = A
    for i in range(len(B)):
        if not( B[i] in A):
            C.append(B[i])
    return C

Le but est maintenant de créer une liste contenant les genres de chaque chanson (puisque la base de données importée ne possède pas les chansons classées par genre, ce qui rend très difficile la création du programme de classification). 
D'abord, on ajoute les données des artistes arrangés dans df_S2 (copie de df_S1, à partir d'ici on travail avec celui-ci), et de même pour la liste des chansons (qui est sauvegardée dans songArtist2), ceci est fait pour mettre à jour les index des artistes et garder le même ordre.

Enfin, en itérant sur les listes travaillées, il est possible de créer la liste songGenres, où chaque élément correspond à la liste des genres d'une chanson (pour chaque chanson). Cela a été fait en attribuant à chaque chanson les genres des artistes qui y participent (données présentes dans df_A).

Il s'agit également d'un algorithme coûteux, alors l'index est aussi imprimé pour voir sa progression.

In [ ]:
df_S2 = new_df_S.reset_index().drop(['index'], axis = 1)
songArtist2 = df_S2['artists'].tolist()

songGenres = []
for i in range(len(songArtist2)):
    print(i)
    auxgen = []
    for j in range(len(songArtist2[i])):
        artistName = songArtist2[i][j]
        strGenre = df_A[df_A['artists'] == artistName]['genres'].tolist()[0]
        auxlist = strGenre.replace("[","").replace("]","").replace("'","").split(",")
        auxgen = union(auxgen,auxlist)
    songGenres.append(auxgen)


Ici on peut voir songGenres

In [ ]:
songGenres

In [ ]:
for i in range(len(songGenres)):
    for j in range(len(songGenres[i])):
        if songGenres[i][j][0] == ' ':
            songGenres[i][j] = songGenres[i][j][1:]
songGenres   

Cette liste, est ajoutée sur la colonne "genres" qui a été précédemment créée sur df_S (df_S2 dans ce cas).

In [ ]:
df_S2['genres'] = songGenres

Ensuite, une liste des genres de chaque artiste est créée (à partir de df_A), en travaillant les données du texte comme auparavant (supprimer les guillemets, les parenthèses, etc.). Ces informations sont sauvegardées dans artistGenres (comme toujours, en sauvegardant les mêmes index que son homologue "dataframe").

In [ ]:
artistGenres = []
for i in range(len(df_A)):
    auxgen = df_A['genres'].tolist()[i].replace("[","").replace("]","").replace("'","").split(",")
    artistGenres.append(auxgen)


In [ ]:
artistGenres

In [ ]:
for i in range(len(artistGenres)):
    for j in range(len(artistGenres[i])):
        if artistGenres[i][j][0] == ' ':
            artistGenres[i][j] = artistGenres[i][j][1:]
artistGenres    

## Construction de l'algorithme.

Une fois que les données sont prêtes, l'algorithme de recommandation des chansons commence à être construit.



On crée d'abord une liste qui contient les noms des colonnes de la "dataframe" qui contiennent des informations techniques sur les chansons, les artistes et les genres. A partir de là, 3 dataframes sont créées (songValues, artistValues et genreValues) qui contiennent uniquement les données techniques de chaque dataframe, dans le même ordre et en conservant les index des données de leurs dataframes précédentes.

In [ ]:
technical = ['acousticness','danceability','energy','instrumentalness',
 'key','liveness','loudness','mode','speechiness',
 'tempo','valence']

songValues = df_S2[technical]
artistValues = df_A[technical]
genreValues = df_G[technical]


Une fonction auxiliaire est créée pour normaliser les vecteurs d'information. Cela est fait parce qu'un des critères à utiliser pour déterminer la proximité entre les chansons sera la distance entre les vecteurs de données techniques de celles-ci, il est donc nécessaire de normaliser les vecteurs car sinon, les données n'auraient pas le même poids.

In [ ]:
# normalizeVectors: list(list(float)) -> nparray(nparray(float))
# ll reçoit une liste python où chaque élément est une liste de nombres réels (float), il retourne un nparray 
# (matrice de même dimension) avec les vecteurs normalisés (chaque vecteur est soustrait de sa moyenne 
# et divisé par son écart-type).

def normalizeVectors(M):
    ncol = len(M.columns)
    nM = np.zeros([len(M),ncol])
    for i in range(ncol):
        nM[:,i] = (np.array(M.iloc[:,i].tolist()) - np.array(M.iloc[:,i].tolist()).mean()) / np.array(M.iloc[:,i].tolist()).std()
    return nM

La fonction est appliquée aux "dataframes" nouvellement créées et enregistrées dans 3 listes différentes.

In [ ]:
song_data = normalizeVectors(songValues)
artist_data = normalizeVectors(artistValues)
genre_data = normalizeVectors(genreValues)

Ici, on peut voir à quoi ressemble song_data:

In [ ]:
song_data

Une fonction est créée pour mesurer la distance entre deux vecteurs.

In [ ]:
# math: nparray(float), nparray(float) -> float
# reçoit deux vecteurs, renvoie la distance entre ceux-ci.

import math
def dist(u,v):
    dif = u - v
    res = 0
    for i in range(len(dif)):
        res += dif[i]**2
    return math.sqrt(res)


Création de la fonction "closest_index":

In [ ]:
# closest_index: array(float), array(array(float)), int -> array(int)
# reçoit un vecteur, une liste de vecteurs et un entier "n", renvoie les indexes des "n" vecteurs de la liste 
# la plus proche du premier vecteur livré.

def closest_index(song,songList,n):
    distances = []
    for i in range(len(songList)):
        distances.append(dist(song,songList[i]))
    closest_index = []
    maxD = max(distances)
    for i in range(n):
        minD = min(distances)
        indexMin = distances.index(minD)
        closest_index.append(indexMin)
        distances[indexMin] = maxD
    return closest_index
    

Pour visualiser comment l'algorithme est créé et comment les fonctions sont utilisées, la chanson "Creep" de Radiohead est utilisée comme exemple.



Tout d'abord, l'index de la chanson est recherché dans df_S2.

In [ ]:
df_S2[df_S2['name']=='Creep'].head(2)

Ensuite, "creep" représente le vecteur normalisé de la chanson, tiré de song_data. Ensuite, les index des 5 chansons les plus proches de Creep de la liste song_data sont enregistrés dans creep_rec (ces index représentent donc directement les index des chansons dans df_S2).

In [ ]:
creep = song_data[24998]
creep_rec = closest_index(creep, song_data, 5)
creep_rec

Les résultats sont affichés. on peut voir qu'il s'agit de résultats relativement cohérents, les genres musicaux n'étant pas si éloignés, mais cela pourrait être bien mieux. 

Pour cela, on procéde à la mise en place d'un filtre par genre.

In [ ]:
df_S2.iloc[creep_rec,:]

L'existence de songGenres et artistGenres est rappelée, et la listeGenres est créée à partir de tous les genres présents dans les artistes.

In [ ]:
#songGenres
#artistGenres
listGenres = []
for i in range(len(artistGenres)):
    listGenres = union(listGenres,artistGenres[i])
listGenres

Création d'une fonction de filtrage par genre musical

In [ ]:
# filter_by_genre: array(str), array(int), int -> array(int)
# Il reçoit une liste des genres d'une chanson, une liste des index des chansons à filtrer et un nombre 
# entier "n". Il renvoie une liste avec les index (par rapport à la deuxième liste livrée) des chansons qui 
# ont au moins "n" genres en commun avec la liste des genres livrés.

def filter_by_genre(genreList,songList_ind,n): #return indexes
    filtered_index = []
    for i in range(len(songList_ind)):
        genresTestedSong = df_S2.iloc[i,-1]
        counter = 0
        for j in range(len(genresTestedSong)):
            if genresTestedSong[j] in genreList:
                counter += 1
        if counter >= n:
            filtered_index.append(songList_ind[i])
    return filtered_index


Pour suivre l'exemple,on prend les genres de Creep, et avec ceux-cion filtre sur toutes les chansons en ne gardant que celles qui n'ont pas plus de 2 genres de différence. Les index des chansons sont sauvegardés dans filteredCreep

Seules les informations pertinentes sont affichées.

In [ ]:
genresCreep = df_S2.iloc[24998,:]['genres']
filteredCreep = filter_by_genre(genresCreep,range(len(df_S2)),len(genresCreep)-2)
filteredCreep

In [ ]:
df_S2.iloc[filteredCreep,[1,12,19]]

Création d'une fonction auxiliaire de sous-liste.

In [ ]:
# subList: list(any), list(int) -> list(any)
# reçoit une liste de quoi que ce soit et une liste d'index de la liste précédente, il renvoie une sous-liste 
# de la première selon les index livrés.

def subList(List,IndexList):
    sub = []
    for i in range(len(IndexList)):
        sub.append(List[IndexList[i]])
    return sub


Ensuite, on rappelle que song_data possède les vecteurs d'information standardisés de toutes les chansons, donc avec la fonction auxiliaire subList on peut prendre seulement (et dans l'ordre désiré) les données des chansons des index de filteredCreep. Ces informations sont stockées dans la liste "closest_to_creep_data":

In [ ]:
closest_to_creep_data = subList(song_data,filteredCreep)
closest_to_creep_data

Ensuite, les index (par rapport à la liste nouvellement créée) des 10 chansons les plus proches de Creep sont sauvegardés (dans une liste appelée creep_index). 

In [ ]:
creep_index = closest_index(creep,closest_to_creep_data,10)
creep_index

Cependant, ces index ne sont pas les vrais (ceux de df_S2), doncon récupere les index réels avec subList dans "real_index".

In [ ]:
real_index = subList(filteredCreep,creep_index)
real_index

Finalement, les résultats sont imprimés, ils sont bien meilleurs.

In [ ]:
df_S2.iloc[real_index,[1,12,19]]

Toutefois, il serait peut-être préférable que le programme ne recommande pas des chansons du même artiste que celui qu'on utilise comme référence, car généralement, une personne qui veut connaître la musique préfère qu'on lui montre quelque chose qui n'est pas du même artiste (cela pourrait être fait par lui-même).



In [ ]:
# filter_artist: str, array(int) -> array(int)
#reçoit un artiste (sous forme de "str") et une liste d'index de chansons, renvoie une sous-liste de la première 
# où ne se trouvent que les index des chansons auxquelles l'artiste livré ne participe pas.

def filter_artist(artist,songList_ind): #return indexes
    filtered_index = []
    for i in range(len(songList_ind)):
        if not(artist in df_S2.iloc[songList_ind[i],1]):
            filtered_index.append(songList_ind[i])
    return filtered_index
    

Pour continuer avec l'exemple, Radiohead est filtré à partir des index (real_index) des chansons précédemment filtrées. Ces nouveaux indices sont enregistrés dans real_index2

In [ ]:
real_index2 = filter_artist('Radiohead',real_index)
real_index2


Enfin, les résultats obtenus sont indiqués (uniquement les informations pertinentes). On peut maintenant vérifier une recommandation acceptable (au moins pour "Creep").

In [ ]:
df_S2.iloc[real_index2,[1,12,19]]

Comme le programme complet fonctionne à travers les index des chansons, il est nécessaire de connaître l'index de la chanson dans laquelle on veut chercher des recommandations. Ensuite, pour faciliter cette tâche, une fonction est créée qui imprime les index et les artistes de toutes les chansons qui sont appelées d'une manière (à mettre par l'utilisateur). C'est également très utile car souvent, différents artistes ont des chansons qui s'appellent les mêmes, donc avec cette méthode on s'assure que l'utilisateur peut choisir la bonne chanson.


In [ ]:
# voirTableauParChanson: str -> None
# reçoit le nom d'une chanson, imprime à l'écran les index et les artistes des chansons qui sont ainsi appelées.

def voirTableauParChanson(nom):
    print(df_S2[df_S2['name'] == nom].iloc[:,[1,12]])
    
voirTableauParChanson('Last Kiss')

## Programme de recommandation musicale

Enfin, un programme interactif est créé, dans lequel tout ce qui est montré ci-dessus est appliqué : d'abord, l'utilisateur doit écrire le nom de la chanson (correctement), ensuite le programme imprime un tableau avec les index et les artistes des chansons qui sont ainsi appelées, puis l'utilisateur doit choisir la chanson qu'il veut et entrer son index. Il sera ensuite demandé à l'utilisateur combien de genres de différence il souhaite que les chansons aient (par rapport à la chanson livrée), par exemple s'il en livre 0, les chansons livrées auront toutes les mêmes genres que la chanson choisie par l'utilisateur, s'il en place 2, les chansons livrées peuvent avoir jusqu'à 2 genres de différence (il est recommandé de ne pas en placer plus de 2). Ensuite, on demande à l'utilisateur jusqu'à combien de recommandations il veut et enfin on lui demande s'il veut éliminer les recommandations qui ont le même artiste.

In [ ]:
songList = df_S2['name'].tolist()

In [ ]:
# searchSong: None -> None
# programme interactif de recommandation de chansons Spotify

def searchSong():
    print("Bienvenue dans le programme interactif de recommandation de musique de Spotify")
    print("Entrez le nom d'une chanson:")
    nom = input()
    if not(nom in songList):
        print("Nom invalide")
    print("Sauvegarder l'index de la chanson souhaitée")
    voirTableauParChanson(nom)
    print("Saisissez l'index souhaité")
    indSong = int(input())
    dataSong = song_data[indSong]
    genresSong = songGenres[indSong]
    artist_of_song = df_S2.iloc[indSong,:]['artists'][0]
    print("Combien de genres musicaux de différance voulez-vous ?")
    ngen = int(input())
    filteredIndex = filter_by_genre(genresSong,range(len(songList)),len(genresSong)-ngen)
    filtered_vectors = subList(song_data,filteredIndex)
    print("Combien de recommandations voulez-vous ?")
    nrec = int(input())
    index2 = closest_index(dataSong,filtered_vectors,nrec)
    index3 = subList(filteredIndex,index2)
    print("Vous souhaitez supprimer des chansons d'un même artiste ? (1 : oui / toute clé : non)")
    memeArt = input()
    if memeArt == '1':
        index4 = filter_artist(artist_of_song,index3)
        print(df_S2.iloc[index4,[1,12]])
    else:
        print(df_S2.iloc[index3,[1,12]])
        
    return
   
    

Pour tester le programme, on peut exécuter la cellule suivante.

In [ ]:
searchSong()